In [1]:
import pandas as pd
from defs import polygon, train_stock_market_model,df_extract_feature, Xy4nn, get_model_performance_result
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler, QuantileTransformer, OneHotEncoder
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Input, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import itertools
from keras import backend as K
from pyswarm import pso

In [2]:
from polygon import RESTClient
stock_name = 'TQQQ'
tf = '60'
unit = 'minute'
name = stock_name+', '+tf+unit

data = polygon(stock_name, tf, unit, '2021-10-01', '2023-10-31').reset_index()
data.rename(columns={'open': 'Open', 'close': 'Close','high':'High','low':'Low','volumn':'Volumn'}, inplace=True)
data.to_csv('data_'+str(name)+'.csv')

data

,Time,Volume,Volume Weighted,Open,Close,High,Low,Time Stamp,Num Items
0,2022-01-03 04:00:00-05:00,108590.0,84.1209,83.990,84.300,84.300,83.9900,1641200400000,1716
1,2022-01-03 05:00:00-05:00,69322.0,84.4253,84.350,84.575,84.580,84.2950,1641204000000,897
2,2022-01-03 06:00:00-05:00,47276.0,84.8254,84.590,84.885,84.945,84.5900,1641207600000,787
3,2022-01-03 07:00:00-05:00,260404.0,84.7407,84.885,84.620,84.885,84.6000,1641211200000,2179
4,2022-01-03 08:00:00-05:00,656510.0,84.5617,84.830,83.950,84.865,83.9350,1641214800000,3879
...,...,...,...,...,...,...,...,...,...
5654,2023-05-31 15:00:00-04:00,15851848.0,34.9090,34.960,34.750,35.160,34.6935,1685559600000,38092
5655,2023-05-31 16:00:00-04:00,770447.0,34.7067,34.740,34.650,34.840,34.6100,1685563200000,2570
5656,2023-05-31 17:00:00-04:00,110714.0,34.6809,34.640,34.690,34.710,34.6400,1685566800000,993
5657,2023-05-31 18:00:00-04:00,188402.0,34.6768,34.690,34.700,34.730,34.6300,1685570400000,782


In [3]:
np.random.seed(0)
# tf.random.set_seed(0)
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [4]:
def train_stock_market_model(X_train, y_train, X_val, y_val, X_test, y_test, cnn_no_node,lstm_no_node, BATCH, kernel_size, LR):
    N = X_train.shape[2]
    #lstm_no_node = 128
    # cnn_no_node = 128
    # kernel_size = 7
    EPOCH = 50
    # LR = 0.0086
    best_mse = 9999999

    object_dict = {
        'feature_columns': feature_columns,
        'scaler': scaler,
        'scaler_label': scaler_label,
        'target_columns': target_columns
    }
    print(X_train.shape)
    # Iterate over combinations of hyperparameters
    # for cnn_no_node, lstm_no_node, kernel_size in list(list(itertools.product(cnn_no_node_list, lstm_no_node_list, kernel_size))):
    #     for j, m, k in itertools.product(range(1,12,2), repeat=3):
            # BATCH = (2 ** j)
            # EPOCH = (10*m)
            # LR = (10**-k)
    input1 = Input(shape=(T, N))
    # Define the model architecture
    x = Conv1D(filters=cnn_no_node, kernel_size=kernel_size, strides=1, padding='same', activation='relu')(input1)
    x = LSTM(lstm_no_node, return_sequences=True)(x)
    x = Dropout(0.6)(x)
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dense(64)(x)
    x = Dropout(0.4)(x)
    out1 = Dense(1, activation='sigmoid', name='class1')(x)

    model = Model(inputs=input1, outputs=[out1])

    model.compile(optimizer=Adam(learning_rate=LR),
                loss='MeanSquaredError',
                metrics='MeanSquaredError')

    try:
        model.load_weights(str(name)+' model.h5')
    except:
        pass
    model.save_weights(str(name)+' model.h5')

    K.set_value(model.optimizer.lr, LR)

    d_k = 256
    d_v = 256
    n_heads = 12
    ff_dim = 256
    lr_decay = ReduceLROnPlateau(monitor='val_loss',
                                patience=10, verbose=0,
                                factor=0.95, min_lr=1e-3, cooldown=0)

    early_stop = EarlyStopping(monitor='val_loss', min_delta=0,
                            patience=30, verbose=0, mode='auto',
                            baseline=None, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train,
                        epochs=EPOCH,
                        batch_size=BATCH,
                        validation_split=0.0,
                        validation_data=(X_val, y_val),
                        shuffle=True, verbose=0,
                        callbacks=[lr_decay, early_stop])

    # Evaluate model performance
    result_train = get_model_performance_result(model, X_train, y_train)
    result_val = get_model_performance_result(model, X_val, y_val)
    result_test = get_model_performance_result(model, X_test, y_test)
    # print(result_val['MSE'][0])
    # Update the best model and MSE if the current model performs better on the validation set
    if best_mse > float(result_val['MSE'][0]):
        best_model = model
        best_mse = float(result_val['MSE'][0])
        best_model = model
    final_loss = history.history['loss'][-1]
    return {'loss': final_loss} 

In [5]:
def display_splits_info(splits_data):
    for i, subset in enumerate(splits_data, 1):
        subset_dates = (subset['Time'].min(), subset['Time'].max())
        
        print(f"Split {i}:")
        print(f"  Data: {subset_dates[0]} - {subset_dates[1]}, Number of records: {len(subset)}")
        print("-" * 60)

In [6]:
def walk_forward_data_splits_by_ratio(data, split_ratio, num_splits):
    """
    Generate walk-forward data splits based on specified split ratio and number of splits.
    """
    # Function to get start indices for each split
    def get_start_indices(data, split_ratio, num_splits):
        total_length = len(data)
        split_length = int(split_ratio * total_length)
        overlap = split_length // 2
        return [i * overlap for i in range(num_splits)]
    
    start_indices = get_start_indices(data, split_ratio, num_splits)
    
    splits_data = []
    for start_index in start_indices:
        end_index = start_index + int(split_ratio * len(data))
        splits_data.append(data.iloc[start_index:end_index])

    return splits_data

In [7]:
# Test the function with the given dataset, split ratio of 0.5 (50%) and 3 splits
splits_by_ratio = walk_forward_data_splits_by_ratio(data, 0.3, 5)

# Use the display function to show the splits information
display_splits_info(splits_by_ratio)

Split 1:
  Data: 2022-01-03 04:00:00-05:00 - 2022-06-06 04:00:00-04:00, Number of records: 1697
------------------------------------------------------------
Split 2:
  Data: 2022-03-21 04:00:00-04:00 - 2022-08-22 04:00:00-04:00, Number of records: 1697
------------------------------------------------------------
Split 3:
  Data: 2022-06-06 04:00:00-04:00 - 2022-11-04 04:00:00-04:00, Number of records: 1697
------------------------------------------------------------
Split 4:
  Data: 2022-08-22 04:00:00-04:00 - 2023-01-24 09:00:00-05:00, Number of records: 1697
------------------------------------------------------------
Split 5:
  Data: 2022-11-04 04:00:00-04:00 - 2023-04-11 09:00:00-04:00, Number of records: 1697
------------------------------------------------------------


In [8]:
def objective_function(params):
    cnn_no_node = int(params[0])
    lstm_no_node = int(params[1])
    BATCH = int(params[2])
    kernel_size = int(params[3])
    LR = params[4]
    # EPOCH = int(params[5])
    result = train_stock_market_model(X_train, y_train, X_val, y_val, X_test, y_test, cnn_no_node, lstm_no_node, BATCH, kernel_size, LR)
    return result['loss']

In [9]:
def custom_pso(objective_func, num_particles, w, c1, c2, max_iterations):
    
    # # cnn_no_node, lstm_no_node, BATCH, kernel_size, LR
    lb = [64, 64, 32, 3, 1e-7] 
    ub = [256, 256, 128, 21, 0.01]


    def pso_objective(x):
        return -objective_func(x)

    best_position, best_score = pso(pso_objective, lb, ub, swarmsize=num_particles, omega=w, phip=c1, phig=c2, maxiter=max_iterations)
    return best_position, best_score

In [10]:
import time
start_time = time.time()

T = 33
# PSO parameter setup
num_particles = 50
w = 0.5  # (weight)
c1 = 1.5  # (cognitive factor)
c2 = 1.5  # (social factor)
max_iterations = 30

best_positions = []
best_scores = []

for i, split in enumerate(splits_by_ratio):
    # Preprocess the data
    split['Time'] = pd.to_datetime(split['Time']).dt.strftime("%Y-%m-%d 00:00:00")
    split['label_1'] = split['Close'].pct_change(1).shift(periods=-1).copy()
    split = df_extract_feature(split).copy()
    split = split.dropna()
    split.reset_index(drop=True, inplace=True)
    
    scaler = MinMaxScaler()
    scaler_label = MinMaxScaler()
    
    feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume',
               'Rel_Open', 'Rel_High', 'Rel_Low', 'Rel_SMA_fast', 'Rel_SMA_slow',
               'CCI', 'ROC', '%R', 'MFI', 'VPT', 'VWAP', 'MACD', 'ADX', 'RSI', 'TSI',
               '%K', '%D', 'Rel_BB_hband', 'Rel_BB_lband', 'Rel_ATR', 'UI', 'CMF',
               'FI']
    target_columns = ['label_1']
    
    train_data = split.iloc[:int(len(split)*0.8),:]
    val_data = split.iloc[int(len(split)*0.8):int(len(split)*0.8+len(split)*0.1),:]
    test_data = split.iloc[int(len(split)*0.8+len(split)*0.1):,:]
    
    # Scale the data
    train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
    train_data[target_columns] = scaler_label.fit_transform(train_data[target_columns])
    
    val_data[feature_columns] = scaler.transform(val_data[feature_columns])
    val_data[target_columns] = scaler_label.transform(val_data[target_columns])
    
    test_data[feature_columns] = scaler.transform(test_data[feature_columns])
    test_data[target_columns] = scaler_label.transform(test_data[target_columns])
    
    # Prepare the input data for the model
    X_train, y_train, train_date = Xy4nn(train_data, feature_columns, target_columns, T)
    X_val, y_val, val_date = Xy4nn(val_data, feature_columns, target_columns, T)
    X_test, y_test, test_date = Xy4nn(test_data, feature_columns, target_columns, T)
    # print(y_train)
    train_date = np.array(train_date).reshape(-1)
    val_date = np.array(val_date).reshape(-1)
    test_data_date = np.array(test_date).reshape(-1)
    
    best_position, best_score = custom_pso(objective_function, num_particles, w, c1, c2, max_iterations)
    best_positions.append(best_position)
    best_scores.append(best_score)
    print(f"Completed PSO for split {i+1}. Best Score: {best_score}")
    print(f"Best_position of PSO :{best_position}")
avg_best_position = np.mean(np.array(best_positions), axis=0)
avg_best_score = np.mean(best_scores)
print(f'Avg of Position: {avg_best_position}, Best Score: {avg_best_score:.4f}')

elapsed_time = time.time() - start_time
print(f"PSO took {elapsed_time:.2f} seconds.")

(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33, 28)
(1296, 33,

In [11]:
A = 92141.21/3600 
A

25.59478055555556

In [ ]:
# Stopping search: maximum iterations reached --> 30
# Completed PSO for split 1. Best Score: -0.3673268258571625
# Best_position of PSO :[6.73824288e+01 1.42366296e+02 3.21672445e+01 3.54230965e+00
#  7.91363939e-03]

In [12]:
# Stopping search: maximum iterations reached --> 30
# Completed PSO for split 2. Best Score: -0.35285481810569763
# Best_position of PSO :[1.41148790e+02 2.21691083e+02 3.20000000e+01 1.76783256e+01
#  1.00000000e-02]

In [13]:
# Stopping search: maximum iterations reached --> 30
# Completed PSO for split 3. Best Score: -0.14588016271591187
# Best_position of PSO :[2.12223076e+02 2.53575617e+02 6.96641187e+01 1.40131080e+01
#  8.63989759e-03]

In [ ]:
# Stopping search: maximum iterations reached --> 30
# Completed PSO for split 4. Best Score: -0.2845204472541809
# Best_position of PSO :[6.40000000e+01 2.56000000e+02 3.20000000e+01 4.50308145e+00
#  1.00000000e-02]

In [ ]:
# Stopping search: maximum iterations reached --> 30
# Completed PSO for split 5. Best Score: -0.1643267720937729
# Best_position of PSO :[2.56000000e+02 6.86571914e+01 6.86559441e+01 2.10000000e+01
#  1.00000000e-07]
# Avg of Position: [1.48150859e+02 1.88458037e+02 4.68974615e+01 1.21473650e+01
#  7.31072740e-03], Best Score: -0.2630
# PSO took 92141.21 seconds.